In [1]:
import os
import requests
from dotenv import load_dotenv
import threading

def main():
    load_dotenv()

    username = os.getenv("ELOQUA_USERNAME")
    password = os.getenv("ELOQUA_PASSWORD")

    base_url = "https://secure.p04.eloqua.com"
    programs_url = f"{base_url}/api/rest/2.0/assets/programs?search=currentStatus='active'"
    program_details_url = f"{base_url}/api/rest/2.0/assets/program/{{}}"

    auth = (username, password)

    response = requests.get(programs_url, auth=auth)

    if response.status_code == 200:
        data = response.json()

        program_ids = []

        search_name = input("Digite um nome para pesquisar nos programas ativos da Eloqua: ")

        def search_in_json(json_obj, search_term):
            if isinstance(json_obj, dict):
                for key, value in json_obj.items():
                    if key == "name" and search_term.lower() in value.lower():
                        return True
                    elif isinstance(value, (dict, list)):
                        if search_in_json(value, search_term):
                            return True
            elif isinstance(json_obj, list):
                for item in json_obj:
                    if search_in_json(item, search_term):
                        return True
            return False

        for element in data["elements"]:
            program_ids.append(element["id"])

        

        def process_program_id(program_id):
            specific_program_url = program_details_url.format(program_id)

            try:
                response = requests.get(specific_program_url, auth=auth)

                if response.status_code == 200:
                    program_details = response.json()

                    for element in program_details["elements"]:
                        if search_in_json(element, search_name):
                            name = element["name"]
                            member_count = element["memberCount"]

                            print(f"Programa ID: {program_id}")
                            print(f"Nome do Elemento: {name}")
                            print(f"Member Count: {member_count}")
                            print()

            except Exception as e:
                print(f"Erro ao processar programa ID {program_id}: {e}")
        
        threads = []

        for program_id in program_ids:
            thread = threading.Thread(target=process_program_id, args=(program_id,))
            threads.append(thread)
            thread.start()

        # start() inicia cria a thread
        for thread in threads:
            thread.join()
        # join() bloqueia a execução do programa até que todas as threads terminem

    else:
        print(f"Erro ao fazer a requisição: {response.status_code}")


if __name__ == "__main__":
    main()


Programa ID: 90
Nome do Elemento: Set Lead Source - FIndLaw DIY
Member Count: 0

Programa ID: 90
Nome do Elemento: FindLAW DIY
Member Count: 0

Programa ID: 90
Nome do Elemento: Lead Source SuperLawyers DB
Member Count: 0

Programa ID: 90
Nome do Elemento: Set Lead Source - SuperLawyers
Member Count: 0

Programa ID: 114
Nome do Elemento: Update Job Level - Law Firm/Govt
Member Count: 0

Programa ID: 114
Nome do Elemento: Is it a solo law firm
Member Count: 0

Programa ID: 114
Nome do Elemento: Global law firm domains
Member Count: 0

Programa ID: 114
Nome do Elemento: EXIT- SALES SUPER LAWYER LEAD
Member Count: 0

Programa ID: 114
Nome do Elemento: SYSTEM: SuperLawyers Contact Missing Country
Member Count: 0

Programa ID: 153
Nome do Elemento: Product is Findlaw
Member Count: 0

Programa ID: 456
Nome do Elemento: FindLaw FL_Master
Member Count: 0

Programa ID: 457
Nome do Elemento: Exit - not a FindLaw contact
Member Count: 12

Programa ID: 488
Nome do Elemento: Super Lawyers Preferenc